In [2]:
#!pip install boto3
#!pip install boto
import os
import boto3
import numpy as np
import pandas as pd
import io

# how to open a log file from s3


client = boto3.client('s3') #low-level functional API

resource = boto3.resource('s3') #high-level object-oriented API
my_bucket = resource.Bucket('adp.spark.app.logs.us-west-1.prd') #subsitute this for your s3 bucket name. 
    

#put the log files's path into a list
files = list(my_bucket.objects.filter(Prefix='logs/aggregates/S3_Logs'))

print("Number of log files : {}".format(len(files)))

files[0:5]

#get the log name
files[2].key.replace('logs/aggregates/S3_Logs/','')

#here the number '2' represents the index of 'files', and you can change to other numbers
obj = files[2].get()
obj

#read a log file
log = obj["Body"].read()
log

type(log)

b =log.decode("utf-8", "strict")
b

type(b)

# extract keywords from log files

In [3]:
client = boto3.client('s3') #low-level functional API

resource = boto3.resource('s3') #high-level object-oriented API
my_bucket = resource.Bucket('adp.spark.app.logs.us-west-1.prd') #subsitute this for your s3 bucket name. 


In [4]:
#put the log files's path and name into a list
files_all = list(my_bucket.objects.filter(Prefix='logs/aggregates'))

print("Number of log files : {}".format(len(files_all)))

Number of log files : 45467


In [5]:
#choose a random sample
import random
sample_number = int(len(files_all)*0.1)
files = random.sample(files_all,sample_number)

In [6]:
############################
#only need to change here! #
############################
# when you run a new list, pls put the old on in the marks, like this:
#keywords_v1 = []
#keywords_v2 = []
keywords = ['exception','kill','fail'] 

In [7]:
#here is Mengyu's version based on shruti's version, we can run this code successfully, but slowly

#from StringIO import StringIO
from io import StringIO
#from datetime import datetime, date
#import time
#import calendar
#import fnmatch
#import csv
#import os




filename = []
results = {}

def getloginfo(keyword):
    ctr = 0
    tempList = []
    for i in range(0,len(files)):
        obj = files[i].get()
        f = obj["Body"].read()
        filename.append(files[i].key.replace('logs/aggregates/S3_Logs/','').replace('logs/aggregates/',''))
        for line in io.StringIO(f.decode("utf-8", "ignore")):
            if keyword in line.lower():
                tempList.append(line)
        results[filename[ctr]] = tempList
        tempList = []
        ctr += 1
    return results


# version2 using map function(not work)
keyword = 'kill'


filename = []
results = {}

def getloginfo2(log):
    tempList = []
    obj = log.get()
    f = obj["Body"].read()
    filename.append(log.key.replace('logs/aggregates/S3_Logs/',''))
    for line in io.StringIO(f.decode("utf-8", "ignore")):
        if keyword in line.lower():
            tempList.append(line)
    results[filename[0]] = tempList
    return results

alpha = list(map(lambda x: getloginfo2(x),files))
            


# Write results to a csv and save it on s3

In [ ]:
# This is the code to write all logs and error msg to csv in this folder (ucd_msba_team) angie's version:
def write_csv(res):
    # change the directory
    import os 
    os.chdir('/home/ec2-user/UCD_MSBA_team/Keywords')
    import csv
    with open(keyword+'_output.csv','w',newline='') as f:             
        w = csv.writer(f)
        w.writerow(['log_id','lines'])
        for key,items in res.items():
            for item in items:
                w.writerow([key,item])



In [ ]:
# run the two functions above
for keyword in keywords:
    res = getloginfo(keyword)
    write_csv(res)